# **Enunciado**

### Autor José Daniel Sarmiento Blanco 2192232

Design a heat exchanger to be used as a crude oil cooler. 120 kg/s of crude oil enters to the heat exchanger at 102°C and leaves at 65°C. The coolant to be used is city water entering the tube side at 21°C with a flow rate of 65 kg/s. Pressure drop for the crude oil is limited to 150 kPa.

In [2]:
caudal_de_crudo             = 120 # kg/s
temperatura_entrada_crudo   = 102 # ºC
temperatura_salida_crudo    = 65  # ºC

temperatura_entrada_agua    = 21  # ºC
caudal_agua                 = 65  # kg/s

presion_limite_crudo        = 150_000 # Pa

In [3]:
from pyfluids import Fluid, FluidsList, Input, Mixture
from core import ThermoFormules
import numpy as np

![](./diagrama.jpeg)

## Suposicion

[presión agua](https://blog.verti.es/hogar/presion-agua-casa/#:~:text=Lo%20habitual%20es%20que%20la,tengamos%20una%20presión%20u%20otra.)


- Considere como material el acero al carbono, k = 60 W/m*k.
- Diámetro de tubo de 3/4 in ( 19 mm exterior y 16 mm interior ).
- Longitud de 4m

In [4]:
k_material                            = 60    # W/ m*k
diametro_interior                     = 16e-3 # m
diametro_exterior                     = 19e-3 # m
longitud                              = 4 #m               

In [5]:
presion_agua = (2) * 101_325 # Pa
presion_crudo = 4136_000 # Pa

In [6]:
temperatura_propiedades_crudo = (temperatura_entrada_crudo + temperatura_salida_crudo) / 2
print(temperatura_propiedades_crudo )#+ 273.15)

cp_propiedades_crudo =  0.83 * 1000  # J/kg k

83.5


$$ \dot{Q} = (\dot{m} *C_{p})_{crudo} * (T_{c1} -T_{c2})$$

In [7]:
calor = caudal_de_crudo * cp_propiedades_crudo * (temperatura_entrada_crudo - temperatura_salida_crudo ) 
calor

3685200.0

In [8]:
water = Fluid(FluidsList.Water)
water.name

Water

### Suponga una temperatura de propiedades de agua de

In [9]:
state_agua = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(27),
)
state_agua.as_dict()

{'compressibility': 0.0014679646038685545,
 'conductivity': 0.6097949184333273,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.5612223522704,
 'dynamic_viscosity': 0.0008508970492459952,
 'enthalpy': 113375.23765002219,
 'entropy': 395.1237635503346,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 113171.88837776403,
 'kinematic_viscosity': 8.538331917406425e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.8331192360367865,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1502.0731022326067,
 'specific_heat': 4180.301802554013,
 'specific_volume': 0.0010034506436439627,
 'surface_tension': None,
 'temperature': 27.0,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [10]:
cp_agua = state_agua.specific_heat
cp_agua

4180.301802554013

$$ \dot{Q} = (\dot{m} *C_{p})_{agua} * (T_{w1} - T_{w2})$$

In [11]:
temperatura_salida_agua = temperatura_entrada_agua + calor / (caudal_agua * cp_agua)
temperatura_salida_agua

34.56250990795588

In [12]:
calor_maximo = cp_propiedades_crudo * (temperatura_entrada_crudo - temperatura_entrada_agua) 
calor_maximo

67230.0

In [53]:
eficiencia = calor_maximo / calor
eficiencia

0.018243243243243244

![](./resistencia.jpeg)

Asuma los coeficiente de trasferencia de calor

$$ \frac{1}{U_{ensuciamiento}} = \frac{1}{h_{0}} + \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + R_{f_{}agua} + R_{f_{}crudo} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} $$

$$ \frac{1}{U_{limpio}} = \frac{1}{h_{0}} + \frac{r_{0}}{r_{i}} \frac{1}{h_{i}} + r_{0} \frac{ln(\frac{r_{0}}{r_{i}} )}{k} $$

In [13]:
h_agua    = 5000 # kj/kg * k
h_crudo   = 800 # kj/kg * k

R_f_crudo = 0.000352 # m2 k/w
R_f_agua  = 0.000176 # m2 k/w

In [14]:
U_ensuciamiento = (
    1/h_crudo + 
    R_f_crudo + 
    (diametro_exterior / 2) * np.log(diametro_exterior/diametro_interior) / k_material +
    (diametro_exterior / diametro_interior) * (1 / h_agua) +
    R_f_agua 
) ** -1
U_ensuciamiento

489.5458406052166

In [15]:
U_no_ensuciamiento = (
    1/h_crudo + 
    (diametro_exterior / 2) * np.log(diametro_exterior/diametro_interior) / k_material +
    (diametro_exterior / diametro_interior) * (1 / h_agua) 
) ** -1
U_no_ensuciamiento

660.192543934851

In [16]:
diferencia_temperatura_1 = temperatura_entrada_crudo - temperatura_salida_agua
diferencia_temperatura_2 = temperatura_salida_crudo - temperatura_entrada_agua
print(diferencia_temperatura_1, diferencia_temperatura_2)

diferencia_log_temperatura = (diferencia_temperatura_2 - diferencia_temperatura_1)/ np.log(diferencia_temperatura_2/diferencia_temperatura_1)
print(diferencia_log_temperatura)

67.43749009204413 44
54.887262211690185


Para el calculo de las el factor de correction se uso las tables del metodo de lmtd del moodle 

In [17]:
R = (temperatura_entrada_crudo - temperatura_salida_crudo) / (temperatura_salida_agua- temperatura_entrada_agua)
R

2.728108606084446

In [18]:
P = 1 / R
P

0.3665543218366454

In [19]:
F = 0.86

In [20]:
diferencia_log_temperatura_corregida =  F * diferencia_log_temperatura

In [21]:
A_f = calor / (U_ensuciamiento * diferencia_log_temperatura_corregida)
A_c = calor / (U_no_ensuciamiento * diferencia_log_temperatura_corregida)

print(A_f, A_c)

159.47686057855407 118.25524914851718


In [22]:
area_ratio =  A_f / A_c - 1
area_ratio

0.34858166319760175

Se toma como referencia las siguientes dimisiones y parámetros de diseño

In [23]:
Cl         = 1
L          = 4   # m
CTP        = 0.93 # One tube pass
PitchRatio = 1.25

In [24]:
diametro_carcasa = 0.637 * (Cl / CTP * A_f * pow(PitchRatio, 2) * diametro_exterior/L)**(1/2) 
print(diametro_carcasa)
diametro_carcasa = 0.7
print(diametro_carcasa)

0.7186271396167857
0.7


In [25]:
numeros_tubos = 0.785 * (CTP/Cl) * (diametro_carcasa)**2 /((PitchRatio * diametro_exterior)**2)
print(numeros_tubos)

634.1930193905818


Se supone un 60 %  del diámetro de la carcasa

In [26]:
B = 0.6 * diametro_carcasa
print('espacio entre baffle', B)
B = 0.5
print('espacio entre baffle', B)

espacio entre baffle 0.42
espacio entre baffle 0.5


## Corrección método de kern

In [27]:
numeros_tubos = 640

In [28]:
T_b = (temperatura_entrada_agua + temperatura_salida_agua) / 2
print(T_b)

27.78125495397794


In [29]:
Pitch_Size              = 0.0254 # m

In [30]:
D_e = 4*(Pitch_Size**2 - np.pi*diametro_exterior**2/4)/(np.pi*diametro_exterior)
D_e

0.024233853930596717

In [31]:
C = Pitch_Size - diametro_exterior
print(C)

0.0063999999999999994


In [32]:
A_s = (0.39 * C * B) / Pitch_Size

print(A_s)

0.04913385826771654


In [33]:
G_s = caudal_de_crudo / A_s

print(G_s)

2442.307692307692


In [34]:
components = [FluidsList.nHexane, FluidsList.nOctane, FluidsList.nDecane]
fractions = [30, 40, 30]
mixture = dict(zip(components, fractions))
mixture

{nHexane: 30, nOctane: 40, nDecane: 30}

In [35]:
state_agua_kern = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(T_b),
)
state_agua_kern.as_dict()

{'compressibility': 0.0014644738083190445,
 'conductivity': 0.6110269720320316,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 996.3433167890219,
 'dynamic_viscosity': 0.000836367771464936,
 'enthalpy': 116641.02661706795,
 'entropy': 405.9901508078167,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 116437.6328705613,
 'kinematic_viscosity': 8.394373278483474e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 5.721638142997106,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1504.02450259156,
 'specific_heat': 4180.0692815493885,
 'specific_volume': 0.001003670103617258,
 'surface_tension': None,
 'temperature': 27.781254953977964,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [36]:
densidad_agua_kern            = state_agua_kern.density
cp_agua_kern                  = state_agua_kern.specific_heat
viscosidad_dinamica_agua_kern = state_agua_kern.dynamic_viscosity
k_agua_kern                   = state_agua_kern.conductivity
prandtl_agua_kern             = state_agua_kern.prandtl

print(densidad_agua_kern, cp_agua_kern, viscosidad_dinamica_agua_kern, k_agua_kern, prandtl_agua_kern)

996.3433167890219 4180.0692815493885 0.000836367771464936 0.6110269720320316 5.721638142997106


In [37]:
crudo = Mixture(
    components,
    fractions=fractions,
)
estado_crudo =  crudo.with_state(
    Input.pressure(101_000),
    Input.temperature(T_b),
)
estado_crudo.as_dict()

{'compressibility': 0.006436783703957838,
 'conductivity': 0.14217900216844762,
 'critical_pressure': 2981833.4687300003,
 'critical_temperature': 298.33744539093857,
 'density': 689.7875540581897,
 'dynamic_viscosity': None,
 'enthalpy': -232835.11238757614,
 'entropy': -574.4950106366606,
 'fluids': [nHexane, nOctane, nDecane],
 'fractions': [30, 40, 30],
 'freezing_temperature': None,
 'internal_energy': -232981.53428204704,
 'kinematic_viscosity': None,
 'max_pressure': 605929364.2590215,
 'max_temperature': 394.3155003716736,
 'min_pressure': 1.5799947781094636,
 'min_temperature': -65.24557909004545,
 'molar_mass': 0.1099927925291343,
 'phase': Liquid,
 'prandtl': None,
 'pressure': 101000,
 'quality': None,
 'sound_speed': 1141.6373570898857,
 'specific_heat': 2235.7944562489474,
 'specific_volume': 0.0014497217210092503,
 'surface_tension': None,
 'temperature': 27.781254953977964,
 'triple_pressure': 1.5799947781094636,
 'triple_temperature': -65.24557909004545,
 'units_system

In [38]:
densidad_crudo_kern            = estado_crudo.density
cp_crudo_kern                  = estado_crudo.specific_heat
viscosidad_dinamica_crudo_kern = 0.001
k_crudo_kern                   = estado_crudo.conductivity
prandtl_crudo_kern             = estado_crudo.prandtl

print(densidad_crudo_kern, cp_crudo_kern, viscosidad_dinamica_crudo_kern, k_crudo_kern, prandtl_crudo_kern)

689.7875540581897 2235.7944562489474 0.001 0.14217900216844762 None


In [39]:
Re = G_s * D_e / viscosidad_dinamica_crudo_kern

print('Reynolds: ', Re)

Reynolds:  59186.52786895735


In [40]:
T_w = 1/2 * (
    (temperatura_entrada_agua + temperatura_salida_agua)/2 +
    (temperatura_entrada_crudo + temperatura_salida_crudo)/2
)
T_w

55.64062747698897

In [41]:
state_average = water.with_state(
    Input.pressure(presion_agua),
    Input.temperature(T_w),
)
state_average.as_dict()

{'compressibility': 0.0013552347549151689,
 'conductivity': 0.6467347002722287,
 'critical_pressure': 22064000.0,
 'critical_temperature': 373.946,
 'density': 985.4258657289612,
 'dynamic_viscosity': 0.0004985548423578504,
 'enthalpy': 233094.944617429,
 'entropy': 776.0879261429805,
 'fraction': 100,
 'freezing_temperature': None,
 'internal_energy': 232889.29748852298,
 'kinematic_viscosity': 5.059283094716093e-07,
 'max_pressure': 1000000000.0,
 'max_temperature': 1726.85,
 'min_pressure': 611.6548008968684,
 'min_temperature': 0.010000000000047748,
 'molar_mass': 0.018015268,
 'name': Water,
 'phase': Liquid,
 'prandtl': 3.2245626960404867,
 'pressure': 202650,
 'quality': None,
 'sound_speed': 1548.102492741577,
 'specific_heat': 4182.963260109866,
 'specific_volume': 0.0010147896810687607,
 'surface_tension': None,
 'temperature': 55.64062747698898,
 'triple_pressure': 611.6548008968684,
 'triple_temperature': 0.010000000000047748,
 'units_system': SIWithCelsiusAndPercents}

In [42]:
viscodiad_dinamica_agua_kern = state_average.dynamic_viscosity

In [43]:
h_0 = 0.36 * (D_e * G_s / viscosidad_dinamica_crudo_kern)**0.55  * (cp_crudo_kern * viscosidad_dinamica_crudo_kern / k_crudo_kern)**(1/3) * (viscosidad_dinamica_crudo_kern / viscodiad_dinamica_agua_kern)**0.14 * k_crudo_kern / D_e
h_0

2458.236773857342

In [44]:
A_tp = np.pi * diametro_interior**2/4 * numeros_tubos/2 
A_tp

0.06433981754551896

In [59]:
u_m = caudal_agua / (densidad_agua_kern * A_tp)
print(u_m)

1.0139686313062197


In [60]:
Re = densidad_agua_kern * u_m * diametro_interior / viscosidad_dinamica_agua_kern
Re

19326.634118694386

### Como el fluido es turbulento se usa correlación de  Grielinki's

In [61]:
f = (1.58*np.log(Re) - 3.28)**-2
f

0.006595445629707155

In [62]:
Nu_b = (f/2)*(Re - 1000)*prandtl_agua_kern/(1 + 12.7*(f/2)**(1/2)*(prandtl_agua_kern**(2/3) - 1))
Nu_b

132.806380665664

In [63]:
h_i = Nu_b * k_agua_kern/diametro_interior
print(h_i)

5071.767540292126


In [64]:
U_f = (
    diametro_exterior / (diametro_interior * h_i) + 
    diametro_exterior / diametro_interior * R_f_crudo + 
    diametro_exterior * np.log(diametro_exterior / diametro_interior) / (2 * k_material) + 
    1/h_i
) ** -1
U_f

1140.8768102763675

In [65]:
U_c = (
    diametro_exterior / (diametro_interior * h_i) + 
    diametro_exterior * np.log(diametro_exterior / diametro_interior) / (2 * k_material) + 
    1 / h_i
) ** -1
U_c

2180.9355424374785

In [66]:
f = np.e**(0.576 - 0.19*np.log(Re))
print(f)

N_b = longitud / B - 1 
print(N_b)

sigma = (viscosidad_dinamica_crudo_kern / viscosidad_dinamica_agua_kern)**0.14
print(sigma)

diferencia_presion = f * G_s ** 2 * (N_b + 1) * 0.39 / (sigma * 2 * densidad_agua_kern * D_e)
diferencia_presion

0.2727620456217141
7.0
1.0253316879520018


30079.98640803056

In [67]:
area = calor /(U_f * diferencia_log_temperatura)
area

58.85057916558362

### da una diferencia de presión inferior a 150kpa (85090) por ende cumple con los requerimientos


![](./matches.png)

### Resultados de la evaluación de diseño

- Material = Acero al carbono k = 60 W/mk
- Longitud del intercambiador = 4 m
- Numero de tubos = 640
- Coeficientes de convección
    - Agua : 5071.76 W/mk
    - Crudo : 2458.236773857342


    

- Diámetro de la carcasa: 0.7 m
- La distribución de los tubos en cuadrada
- Un Pitch_Size de 0.0254 m

- Temperaturas de trabajo
    - Agua :
        - Entrada : 21 º C
        - Salida  : 34.5 º C
    - Crudo :
        - Entrada : 102 ºC
        - Salida  : 65  ºC


## Referencias
- Abdulagatov, I.M., Abdulagatova, Z.Z., Grigor’ev, B.A. et al. Thermal Diffusivity, Heat Capacity, and Thermal Conductivity of Oil Reservoir Rock at High Temperatures. Int J Thermophys 42, 135 (2021). https://doi.org/10.1007/s10765-021-02878-x
- Seguros, V. (2021) Presión del Agua en Casa: Problemas habituales: Blog Verti, Verti Blog. Available at: https://blog.verti.es/hogar/presion-agua-casa/#:~:text=Lo%20habitual%20es%20que%20la,tengamos%20una%20presión%20u%20otra. (Accessed: 13 October 2023). 
- Bell, I.H. et al. (2014) ‘Pure and pseudo-pure fluid thermophysical property evaluation and the open-source Thermophysical Property Library coolprop’, Industrial &amp;amp; Engineering Chemistry Research, 53(6), pp. 2498–2508. doi:10.1021/ie4033999. 